In [2]:
import pygal
import pandas as pd
import streamlit as st
from datetime import datetime

In [37]:
# Definindo a data mínima e máxima
data_minima = pd.to_datetime('2023-01-01')
data_maxima = pd.to_datetime('2024-05-23')

    
# Adicionar seletor de períodos na coluna à esquerda
start_date_ = st.sidebar.date_input("Data de início", data_minima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #"DD-MM-YYYY"

end_date_ = st.sidebar.date_input("Data de término", data_maxima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #+ pd.DateOffset(days=1) #"DD-MM-YYYY"

start_date = start_date_.strftime('%Y-%m-%d') #'%d-%m-%Y'
end_date = end_date_.strftime('%Y-%m-%d')

periodo = end_date_ - start_date_

# Periodo anteior para gráficos com comparações
start_date_b4 = start_date_ - pd.DateOffset(days = periodo.days+1)

end_date_b4 = end_date_ - pd.DateOffset(days = periodo.days+1)

In [38]:
import csv

def encontrar_frase_em_csv_meta(nome_arquivo, frase_procurada):
    try:
        with open(nome_arquivo, 'r', newline='', encoding='utf-16') as arquivo_csv:
            leitor_csv = csv.reader(arquivo_csv)
            
            for numero_linha, linha in enumerate(leitor_csv, start=1):
                if frase_procurada in linha:
                    return numero_linha

        # Se a frase não for encontrada em nenhuma linha
        return -1

    except FileNotFoundError:
        print(f'O arquivo {nome_arquivo} não foi encontrado.')
        return -1

In [39]:
final_alcanceFB = encontrar_frase_em_csv_meta('tabelas/meta/alcance.csv', 'Alcance do Instagram')

dados_FB_alcance = pd.read_csv('tabelas/meta/alcance.csv',skiprows=2,encoding='utf-16',nrows=final_alcanceFB-5)



In [40]:
# Função para tentar múltiplos formatos
def try_parsing_date(text):
    for fmt in ("%Y-%m-%dT%H:%M:%S.%fZ", "%Y-%m-%d %H:%M:%S"):
        try:
            return pd.to_datetime(text, format=fmt)
        except ValueError:
            continue
    return pd.NaT

In [41]:
dados_FB_alcance['Data'] = dados_FB_alcance['Data'].apply(try_parsing_date)
dados_FB_alcance = dados_FB_alcance.dropna(subset=['Data'])

In [42]:
# Recebendo dados de 01/01/2022 até o momento
# alcance FB: 28/05/2022
# alcance IG: 28/05/2022
# visitas FB: 28/05/2022
# visitas IG: 28/05/2022
# seguidores FB: 01/01/2022
# seguidores IG: 31/08/2023
def filtrosDatasFACEBOOK(start_date, end_date, start_date_b4, end_date_b4):
    dados_FB_alcance_FILTRADAS = dados_FB_alcance.copy()
    
    dados_FB_alcance_FILTRADAS = dados_FB_alcance_FILTRADAS.loc[(dados_FB_alcance_FILTRADAS['Data']>start_date) & (dados_FB_alcance_FILTRADAS['Data']<end_date)]
    
    dados_FB_alcance_ANTERIOR = dados_FB_alcance.copy()
    
    dados_FB_alcance_ANTERIOR = dados_FB_alcance_ANTERIOR.loc[(dados_FB_alcance_ANTERIOR['Data']>start_date_b4) & (dados_FB_alcance_ANTERIOR['Data']<end_date_b4)]
    
    return dados_FB_alcance_ANTERIOR,dados_FB_alcance_FILTRADAS

In [52]:
import plotly.express as px

dados_FB_alcance_ANTERIOR,dados_FB_alcance_FILTRADAS = filtrosDatasFACEBOOK(start_date, end_date, start_date_b4.strftime('%Y-%m-%d'), end_date_b4.strftime('%Y-%m-%d'))

fig = px.line(dados_FB_alcance_FILTRADAS, x=map(lambda d: pd.to_datetime(d).strftime('%d-%m-%Y'), dados_FB_alcance_FILTRADAS['Data']), y="Primary", title='Life expectancy in Canada')
fig.show()

TypeError: object of type 'map' has no len()

In [54]:
test = map(lambda d: pd.to_datetime(d).strftime('%d-%m-%Y'), dados_FB_alcance_FILTRADAS['Data'])

print(test)